# [E-14] Movielens

> 목차
-


**1. 개요**  
    
    1.1 프로젝트 과정 및 목표  

**2. 루브릭 평가기준**  

**3. Project**
    
    3.1 데이터 준비하기    
    3.2 데이터 전처리  
    3.3 데이터 분리 및 학습시키기  
    3.4 데이터 테스트
**4. 회고**
    
    4.1 결과 평가  
    4.2 어려웠던 점
    4.3 참조
    

---
> **1. 개요**
-

> **1.1 프로젝트 과정 및 목표**

**<프로젝트 과정>**
1. 데이터 준비와 전처리
2. 데이터 분석
3. 선호 데이터 ratings에 추가
4. CSR matrix만들기
5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련
6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악
7. 내가 좋아하는 영화와 비슷한 영화를 추천
8. 내가 가장 좋아할 만한 영화들을 추천

---
> **2. 루브릭 평가기준**
-

|평가문항 | 상세기준|
|:---------|:---------|
|1. CSR matrix가 정상적으로 만들어졌다. | 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. | 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.| MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

---
> **3. Project**
-

> 3.1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
data = ratings[['user_id', 'movie_id', 'counts']]

In [6]:
data

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


> 3.2. 데이터 분석

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies['title'][0][:-7]
title2 = []

In [9]:
for i in range(3883):
     title2.append(movies['title'][i][:-7])

In [10]:
title2

['Toy Story',
 'Jumanji',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'Sudden Death',
 'GoldenEye',
 'American President, The',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Cutthroat Island',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Ace Ventura: When Nature Calls',
 'Money Train',
 'Get Shorty',
 'Copycat',
 'Assassins',
 'Powder',
 'Leaving Las Vegas',
 'Othello',
 'Now and Then',
 'Persuasion',
 'City of Lost Children, The',
 'Shanghai Triad (Yao a yao yao dao waipo qiao)',
 'Dangerous Minds',
 'Twelve Monkeys',
 'Wings of Courage',
 'Babe',
 'Carrington',
 'Dead Man Walking',
 'Across the Sea of Time',
 'It Takes Two',
 'Clueless',
 'Cry, the Beloved Country',
 'Richard III',
 'Dead Presidents',
 'Restoration',
 'Mortal Kombat',
 'To Die For',
 'How to Make an American Quilt',
 'Seven (Se7en)',
 'Pocahontas',
 'When Night Is Falling',
 'Usual Suspects, The',
 'Guardian Angel',
 'Mighty Aphrod

In [11]:
movies.drop(['title'], axis=1, inplace=True)
movies2 = movies.assign(Title=title2)

In [12]:
movies2

,movie_id,genre,Title
0,1,Animation|Children's|Comedy,Toy Story
1,2,Adventure|Children's|Fantasy,Jumanji
2,3,Comedy|Romance,Grumpier Old Men
3,4,Comedy|Drama,Waiting to Exhale
4,5,Comedy,Father of the Bride Part II
...,...,...,...
3878,3948,Comedy,Meet the Parents
3879,3949,Drama,Requiem for a Dream
3880,3950,Drama,Tigerland
3881,3951,Drama,Two Family House


## movie_id를 영화제목으로 치환하기 위한 딕셔너리 제작

In [13]:
id_to_movie = {k+1:v for k,v in enumerate(movies2['Title'])}

In [14]:
id_to_movie

{1: 'Toy Story',
 2: 'Jumanji',
 3: 'Grumpier Old Men',
 4: 'Waiting to Exhale',
 5: 'Father of the Bride Part II',
 6: 'Heat',
 7: 'Sabrina',
 8: 'Tom and Huck',
 9: 'Sudden Death',
 10: 'GoldenEye',
 11: 'American President, The',
 12: 'Dracula: Dead and Loving It',
 13: 'Balto',
 14: 'Nixon',
 15: 'Cutthroat Island',
 16: 'Casino',
 17: 'Sense and Sensibility',
 18: 'Four Rooms',
 19: 'Ace Ventura: When Nature Calls',
 20: 'Money Train',
 21: 'Get Shorty',
 22: 'Copycat',
 23: 'Assassins',
 24: 'Powder',
 25: 'Leaving Las Vegas',
 26: 'Othello',
 27: 'Now and Then',
 28: 'Persuasion',
 29: 'City of Lost Children, The',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao)',
 31: 'Dangerous Minds',
 32: 'Twelve Monkeys',
 33: 'Wings of Courage',
 34: 'Babe',
 35: 'Carrington',
 36: 'Dead Man Walking',
 37: 'Across the Sea of Time',
 38: 'It Takes Two',
 39: 'Clueless',
 40: 'Cry, the Beloved Country',
 41: 'Richard III',
 42: 'Dead Presidents',
 43: 'Restoration',
 44: 'Mortal Kombat',

### Title안에 년도가 있는데 일일히 확인하기 힘들어서 제거 했다.

In [15]:
movies2['movie_id'].nunique()

3883

In [16]:
movies2['genre'].nunique()

301

In [17]:
# 인기 많은 영화
movie_count = data.groupby('movie_id')['user_id'].count()
best = movie_count.sort_values(ascending=False).head(30)

In [18]:
best

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [19]:
# best작품들을 Title로 변환
[id_to_movie[i] for i in best.index]

['Hairspray',
 'Little Princess, A',
 'GoodFellas',
 'Once Upon a Time in America',
 'Sleeping Beauty',
 'Batman',
 'Pinocchio',
 'Army of Darkness',
 'Koyaanisqatsi',
 'Mommie Dearest',
 'King of the Hill',
 'Cabaret Balkan (Bure Baruta)',
 'Hellraiser: Bloodline',
 'Taxi Driver',
 'Kicked in the Head',
 'Second Best',
 'Alien',
 'Trick or Treat',
 'Gang Related',
 'Suture',
 'Death in Brunswick',
 'Big Sleep, The',
 'Palookaville',
 'Ten Benny',
 'Masque of the Red Death, The',
 'Pushing Hands',
 'I Like It Like That',
 'Stand by Me',
 'Toy Story',
 'Getting Even with Dad']

In [20]:
# 유저별 몇 개의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [21]:
# 유저별 평점 중앙값에 대한 통계
user_median = data.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

> 3.3. 선호 데이터 ratings에 추가

## 이 다음 연산에서 값들을 다시 인덱싱 할것이기 때문에 숫자로 넣어둔다.

In [22]:
# 데이터셋에 임의의 5개를 선호 작으로 추출
my_favorite = ['1' , '2' ,'3' ,'4' ,'5']

# 'young'이라는 user_id가 위 영화의 평점을 5점 줬다고 가정하겠습니다.
my_movie = pd.DataFrame({'user_id': ['6700']*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_movie)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6700,1,5
1,6700,2,5
2,6700,3,5
3,6700,4,5
4,6700,5,5


In [23]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [24]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,3628,5
1,6039,3629,5
2,6039,3630,5
3,6039,3631,5


> 3.4. CSR matrix만들기

In [25]:
num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

In [26]:
num_user

6040

In [27]:
num_movie

3633

In [28]:
from scipy.sparse import csr_matrix

csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

> 3.5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [29]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [30]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=5000, regularization=0.01, use_gpu=False, iterations=1, dtype=np.float32)

'''
factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
use_gpu : GPU를 사용할 것인지
iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
'''

'\nfactors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지\nregularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지\nuse_gpu : GPU를 사용할 것인지\niterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지\n'

In [31]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [32]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/1 [00:00<?, ?it/s]

>3.6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [33]:
# id 를 6700로 했던걸 young으로 치환, 1 = Toy Story
# 토이스토리 = 선호영화
young, Toy_Story = user_to_idx['6700'], movie_to_idx['1']
young_vector, Toy_Story_vector = als_model.user_factors[young], als_model.item_factors[Toy_Story]

In [34]:
# young과 Toy Story를 내적하는 코드
np.dot(young_vector, Toy_Story_vector)

0.80200183

In [35]:
other = movie_to_idx['2']
other_vector = als_model.item_factors[other]

In [36]:
np.dot(young_vector, other_vector)

0.81627613

>3.7. 내가 좋아하는 영화와 비슷한 영화를 추천

In [37]:
favorite_artist = '1'
movie_id = movie_to_idx[favorite_artist]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3628, 1.0),
 (3201, 0.774033),
 (3579, 0.77151936),
 (2494, 0.77125555),
 (3503, 0.7709636),
 (2797, 0.77063274),
 (2001, 0.77058524),
 (3145, 0.770533),
 (1696, 0.7703232),
 (3269, 0.7696895),
 (3623, 0.7696408),
 (3110, 0.76947033),
 (3214, 0.7694541),
 (3038, 0.7692986),
 (3204, 0.769205)]

In [38]:
def get_similar_movie(movie_name: str):
    artist_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [id_to_movie[i[0]] for i in similar_movie]
    return similar_movie


In [39]:
get_similar_movie(3017)

['Night of the Creeps',
 'Forever Young',
 'Running Free',
 'Bandits',
 'Time Code',
 'Sugar Town',
 'Trip to Bountiful, The',
 'Batman: Mask of the Phantasm',
 'Wag the Dog',
 "I'll Never Forget What's 'is Name"]

>3.8. 내가 가장 좋아할 만한 영화들을 추천

In [42]:
user = user_to_idx['6700']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1066, 0.09832441),
 (930, 0.09112028),
 (1399, 0.0909337),
 (2598, 0.08474364),
 (2105, 0.084209874),
 (1595, 0.083770305),
 (1132, 0.082520574),
 (1194, 0.08108629),
 (1932, 0.08057362),
 (3246, 0.08025843),
 (3117, 0.07889505),
 (1791, 0.0785154),
 (2777, 0.0764588),
 (1124, 0.075723745),
 (1681, 0.07571459),
 (1095, 0.07556508),
 (730, 0.07494177),
 (2929, 0.07276884),
 (1129, 0.072729275),
 (2229, 0.07270797)]

In [43]:
[id_to_movie[i[0]] for i in movie_recommended]

['Victor/Victoria',
 'Mark of Zorro, The',
 'Murder at 1600',
 'It Conquered the World',
 'Navigator: A Mediaeval Odyssey, The',
 'How to Be a Player',
 'When We Were Kings',
 'Wings of Desire (Der Himmel über Berlin)',
 'Lethal Weapon',
 'Big Trees, The',
 'Snow Falling on Cedars',
 'Taste of Cherry',
 'White Boys',
 'Everything Relative',
 'Jackie Brown',
 'Bird of Prey',
 'Garcu, Le',
 'Being John Malkovich',
 'Line King: Al Hirschfeld, The',
 'What Dreams May Come']

> **4. 회고**
-

> 4.1 결과 평가

여태까지 노드를 진행하면서 대부분의 노드들이 훈련이 최소 열번은 하니깐 이번에도 10번, 15번 100번등 여러번의 시도를 해봤는데 오히려 csr 내적수치가 0으로 수렴해가서 고민하던차에 2조에 휴민님과 3조에 다혜님이랑 얘기하다가 훈련 횟수가 오히려 안좋은 결과를 불러 올 수 있다는 얘기를 들어서 바로 1번으로 고쳐서 대략적으로 0.8의 값을 얻을 수 있었습니다. 비록 1에 극한까지 수렴하진 않았지만 그 전보다 유의미한 값을 얻었다고 생각합니다.

> 4.2 어려웠던 점

해외영화라서 나온 영화들이 관련있는 영화인지를 알 수가 없어서 제대로 추천이 됐는지 파악하기 힘들었고 훈련을 오히려 적게해야된다는게 이해가 안되서 이부분에 대해서 공부 해야할 필요성을 느꼈습니다.

> 4.3  참조

2조 휴민님과 3조 다혜님한테 반복수가 클 수록 오히려 결과가 나빠진다는 힌트를 얻을 수 있었습니다.